In [0]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
from mlxtend.evaluate import permutation_test

sns.set_style({'font.family': 'Liberation Serif'})
palette = sns.color_palette("tab10", n_colors=10)
color_code = {'Single': palette[1],
              'Multitask': palette[0],
              'Projection': palette[2],
              'Unweighted': palette[3],
              'Weighted': palette[4],
              'Orthogonal': palette[5],
              'AATW': palette[7]}

order = ['Single task',
         'Multitask',
         'Projection',
         'Projection (exponential smoothing)',
         'Projection (variable-wise)',
         r'Projection ($\alpha$ = 1)',
         r'Projection ($\alpha$ = 0.01)',
         'Unweighted cosine',
         'Weighted cosine',
         'Orthogonal',
         r'AATW ($\beta$ = 100)',
         r'AATW ($\beta$ = 1000)',
         r'AATW ($\beta$ = 10000)',
         r'AATW ($\beta$ = 100000)']

#Experiment1: CelebA

In [0]:
results_file = "experiment-1.csv"
df_all = pd.read_csv(results_file)
df_all['variant'] = (df_all['alpha'] == 0.01)*1 + (df_all['overall'] == False)*2
df_all['variant'] = df_all['variant'].apply(lambda x: '' if x == 0 else 'exponential smoothing' if x == 1 else 'variable-wise' if x == 2 else 'exponential smoothing, variable-wise')
df = df_all
df = df[df['mode'] != 'Unweighted cosine']
df = df[df['mode'] != 'Weighted cosine']
df = df[df['mode'] != 'Orthogonal']

df['mode2'] = df['mode']
for i in df.index:
  if df.loc[i, 'mode'] == 'Projection':
    if df.loc[i, 'variant'] != '':
      df.loc[i, 'mode2'] = df.loc[i, 'mode2'] + ' (' + df.loc[i, 'variant'] + ')'

lambdas = list(np.sort(np.unique(df['lam'])))
single_task_df = df[df['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df = pd.concat([df, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df = pd.concat([df, single_task_df], ignore_index=True)

hue_order = [x for x in order if x in np.unique(df['mode2'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])

matplotlib.pyplot.gcf().set_size_inches(3.25, 4.5)
ax = plt.axes()
sns.lineplot(x='lam', y='test_auc', style='mode2', hue='mode2', hue_order=hue_order, style_order=hue_order, data=df.query('lam > 0'), ax=ax, legend='full')

plt.xscale('log')
legend_labels = list(np.unique(df['mode2']))
handles, labels = ax.get_legend_handles_labels()

handles, labels = ([handles[i] for i in range(len(labels)) if labels[i] in legend_labels],
                   [labels[i] for i in range(len(labels)) if labels[i] in legend_labels])
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.legend(handles, labels, fontsize=9, ncol=1, frameon=False)
plt.ylabel('Testing AUC (higher is better)', fontsize=9)
plt.ylim((0.8365,0.852))
plt.xlabel(r'$\lambda$ (log scale)', fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=9)
sns.despine()
plt.savefig('celeba.pdf', bbox_inches = 'tight')

In [0]:
a = df.query('lam == 3000').query('mode == "Multitask"')['test_auc']
b = df.query('lam == 10000').query('mode == "Projection"').query('alpha == 1').query('overall == True')['test_auc']
result = permutation_test(a, b, func='x_mean < y_mean', method='approximate', num_rounds=100000, seed=0)
print('%.1f%%'% (100*result))

In [0]:
results_file = "experiment-1.csv"
df_all = pd.read_csv(results_file)
df0 = df_all[df_all['mode'] == 'Single task']
lambdas = list(np.unique(df_all['lam']))
df0.loc[:, 'lam'] = lambdas[1]
single_task_df = df0.copy()
single_task_df.loc[:, 'lam'] = lambdas[-1]
df0 = pd.concat((df0, single_task_df))
df0 = pd.concat((df0, df_all[df_all['mode'] == 'Multitask']))

hue_order = [x for x in order if x in np.unique(df_all['mode'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])

def chart_celeba(df, legend='brief', xlabel=False, title='', ax=None):
  if ax is None:
    ax = plt.gca()
  sns.lineplot(x='lam',
               y='test_auc',
               style='mode',
               hue='mode',
               hue_order=hue_order,
               style_order=hue_order,
               data=df,
               legend=legend,
               ax=ax)
  ax.set_xscale('log')
  ax.xaxis.set_major_locator(matplotlib.ticker.FixedLocator(lambdas))
  ax.xaxis.set_minor_locator(matplotlib.ticker.NullLocator())
  if legend is not False:
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, frameon=False)
  ax.set_xlabel(r'$\lambda$ (log scale)' if xlabel else '', fontsize=9)
  ax.set_ylabel('Testing AUC (higher is better)', fontsize=9)
  ax.tick_params(axis='both', which='major', labelsize=9)
  ax.set_title(title, fontsize=9)
  sns.despine()

fig, axes = plt.subplots(figsize=(6.75, 3), ncols=5, sharey=True)

combinations = [(overall, alpha) for overall in [True, False] for alpha in [1, 0.01]]
for i in range(len(combinations)):
  overall, alpha = combinations[i]
  df = df_all[df_all['mode'] != 'Multitask']
  df = df[df['mode'] != 'Single task']
  df = df_all[df_all['overall'] == overall]
  df = df[df['alpha'] == alpha]
  df = pd.concat((df, df0))
  legend = 'brief' if i == 3 else None
  xlabel = (i == 2)
  title = '\n'.join(([r'$\alpha$ = 0.01' if alpha == 0.01 else 'No smoothing', '' if overall else 'Variable-wise']))
  chart_celeba(df, legend=legend, xlabel=xlabel, title=title, ax=axes[i])

handles, labels = axes[-2].get_legend_handles_labels()
labels = [s.capitalize() for s in labels]
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
labels, handles = labels[1:], handles[1:]
axes[-2].clear()

chart_celeba(df, legend=False, xlabel=xlabel, title=title, ax=axes[i])
axes[-1].axis('off')
_ = axes[-1].legend(handles, labels, loc='center left', frameon=False, fontsize=9)
plt.ylim((0.83,0.855))
plt.savefig('celeba2.pdf', bbox_inches = 'tight')

In [0]:
a = df_all.query('lam == 10000').query('mode == "Projection"').query('alpha == 1').query('overall == False')['test_auc']
b = df_all.query('lam == 10000').query('mode == "Orthogonal"').query('alpha == 0.01').query('overall == False')['test_auc']
result = permutation_test(a, b, func='x_mean < y_mean', method='approximate', num_rounds=100000, seed=0)
print('%.2f%%'% (100*result))

In [0]:
df_all.pivot_table(index=['mode', 'alpha'], columns=['lam', 'overall'], values='test_auc', aggfunc=len).applymap(lambda x: int(np.nan_to_num(x)))

In [0]:
results_file = "experiment-1.csv"
results_file2 = "experiment-1a.csv"
df = pd.read_csv(results_file)
df2 = pd.read_csv(results_file2)

df = df.query('overall == True').query('alpha == 1.00')
df = df.query('mode == "Single task" or mode == "Multitask"')

lambdas = list(np.sort(np.unique(df['lam'])))
single_task_df = df[df['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df = pd.concat([df, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df = pd.concat([df, single_task_df], ignore_index=True)

df2['mode'] = ''
for i in df.index:
  df2.loc[i, 'mode'] = r'AATW ($\beta$ = ' + str(int(df2.loc[i, 'beta'])) + ')'
df = pd.concat((df[['lam', 'mode', 'test_auc']],
                df2[['lam', 'mode', 'test_auc']]))
df = df.query('lam > 0')

matplotlib.pyplot.gcf().set_size_inches(3.25, 5.25)
ax = plt.axes()
hue_order = [x for x in order if x in np.unique(df['mode'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])
sns.lineplot(x="lam", y="test_auc", hue='mode', hue_order=hue_order, style='mode', style_order=hue_order, data=df, ax=ax)
plt.xscale('log')
plt.ylabel('Testing AUC (higher is better)', fontsize=9)
plt.xlabel(r'Initial $\lambda$ (log scale)', fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=9)
handles, labels = ax.get_legend_handles_labels()
labels = [s.replace('000)', ',000)') for s in labels]
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.legend(handles[1:], labels[1:], fontsize=9, frameon=False, ncol=1)
plt.ylim((0.835,0.851))
sns.despine()
plt.savefig('AATW.pdf', bbox_inches = 'tight')

In [0]:
results_file = "experiment-1.csv"
results_file2 = "experiment-1b.csv"
df = pd.read_csv(results_file)
df2 = pd.read_csv(results_file2)

df = df.query('overall == True').query('alpha == 1.00')
df = df.query('mode == "Single task" or mode == "Multitask"')

lambdas = list(np.sort(np.unique(df['lam'])))
single_task_df = df[df['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df = pd.concat([df, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df = pd.concat([df, single_task_df], ignore_index=True)

df2['mode'] = ''
for i in df.index:
  df2.loc[i, 'mode'] = r'AATW ($\beta$ = ' + str(int(df2.loc[i, 'beta'])) + ')'
df = pd.concat((df[['lam', 'mode', 'test_auc']],
                df2[['lam', 'mode', 'test_auc']]))
df = df.query('lam > 0')

matplotlib.pyplot.gcf().set_size_inches(3.25, 5.25)
ax = plt.axes()
hue_order = [x for x in order if x in np.unique(df['mode'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])
sns.lineplot(x="lam", y="test_auc", hue='mode', hue_order=hue_order, style='mode', style_order=hue_order, data=df, ax=ax)
plt.xscale('log')
plt.ylabel('Testing AUC (higher is better)', fontsize=9)
plt.xlabel(r'Initial $\lambda$ (log scale)', fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=9)
handles, labels = ax.get_legend_handles_labels()
labels = [s.replace('000)', ',000)') for s in labels]
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.legend(handles[1:], labels[1:], fontsize=9, frameon=False, ncol=1)
plt.ylim((0.835,0.851))
sns.despine()
plt.savefig('AATW2.pdf', bbox_inches = 'tight')

#Experiment 2: Synthetic dataset

In [0]:
results_file = "experiment-2.csv"
df_all = pd.read_csv(results_file)
df = df_all[df_all['mode'] != 'Unweighted cosine']
df = df[df['mode'] != 'Weighted cosine']
df = df[df['mode'] != 'Orthogonal']

lambdas = list(np.sort(np.unique(df['lam'])))
single_task_df = df[df['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df = pd.concat([df, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df = pd.concat([df, single_task_df], ignore_index=True)

In [0]:
matplotlib.pyplot.gcf().set_size_inches(3.25, 3.25)
ax = plt.axes()
df = df.query('situation == "Same task"')
df = df[(df['mode'] != 'Projection') + (df['alpha'] == 0.01)]
df['mode2'] = df['mode']
for i in df.index:
  if df.loc[i, 'mode'] == 'Projection':
    if not df.loc[i, 'overall']:
      df.loc[i, 'mode2'] = df.loc[i, 'mode2'] + ' (variable-wise)'


hue_order = [x for x in order if x in np.unique(df['mode2'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])
sns.lineplot(x="lam", ax=ax, y="test_loss", hue='mode2', hue_order=hue_order, style='mode2', style_order=hue_order, data=df.query('lam > 0'))
plt.xscale('log')
handles, labels = ax.get_legend_handles_labels()
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.legend(handles[1:], labels[1:], fontsize=9, frameon=False, ncol=1)
plt.ylabel('Testing loss (lower is better)', fontsize=9)
plt.ylim((0.405,0.475))
plt.xlabel(r'$\lambda$ (log scale)', fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=9)
sns.despine()
plt.savefig('synthetic2.pdf', bbox_inches = 'tight')

In [0]:
def remove_useless_zero(x):
  return x if x[-2:] != '.0' else x[:-2]

results_file = "experiment-2.csv"
df_all = pd.read_csv(results_file)
df = df_all[df_all['overall']]
df = df[df['mode'] != 'Unweighted cosine']
df = df[df['mode'] != 'Weighted cosine']
df = df[df['mode'] != 'Orthogonal']

single_task_df = df[df['mode'] == 'Single task'].copy()
lambdas = list(np.unique(df['lam']))
ticks = [remove_useless_zero(str(x)) for x in lambdas]
for lam in [lambdas[1], lambdas[-1]]:
  single_task_df.loc[:, 'lam'] = lam
  df = pd.concat([df, single_task_df], ignore_index=True)
df = df[df['lam'] > 0]

hue_order = [x for x in order if x in np.unique(df['mode'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])

def chart_synthetic(df, legend='brief', hue='mode', ax=None):
  if ax is None:
    ax = plt.gca()

  sns.lineplot(x='lam',
               y='test_loss',
               style=hue,
               style_order=hue_order,
               hue=hue,
               hue_order=hue_order,
               data=df,
               legend=legend,
               ax=ax)
  ax.set_xscale('log')
  ax.xaxis.set_major_locator(matplotlib.ticker.FixedLocator(lambdas))
  ax.xaxis.set_minor_locator(matplotlib.ticker.NullLocator())
  ax.xaxis.set_major_formatter(matplotlib.ticker.FixedFormatter(ticks))
  ax.tick_params(axis='both', which='major', labelsize=9)
  sns.despine()

fig, axes = plt.subplots(figsize=(6.75, 3.5), ncols=4, nrows=3, sharey=True, sharex=True)
axes[0][1].set_title('Similar task', fontsize=9)

situations = ['Unrelated task', 'Similar task', 'Same task']
alphas = list(np.unique(df['alpha']))[::-1]

df1 = df.query('mode == "Single task"')
for column in range(3):
  df2 = pd.concat([df1, df.query('mode == "Multitask"').query('situation == "' + situations[column] + '"')])
  for row in range(3):  
    df3 = pd.concat([df2, df.query('mode != "Multitask"')
                            .query('mode != "Single task"')
                            .query('situation == "' + situations[column] + '"')
                            .query('alpha == '+str(alphas[row]))])
    legend = False if ((row < 2) | (column < 2)) else 'brief'
    chart_synthetic(df3, legend=legend, ax=axes[row][column])
handles, labels = axes[2][2].get_legend_handles_labels()
axes[2][2].clear()
chart_synthetic(df3, legend=False, ax=axes[2][2])
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
_ = axes[1][3].legend(handles[1:], labels[1:], loc='center', frameon=False, fontsize=9)
for row in range(3):
  axes[row][3].axis('off')
for column in range(3):
  axes[0][column].set_title(situations[column], fontsize=9)
  _ = axes[2][column].set_xlabel(r'$\lambda$ (log scale)' if column == 1 else '', fontsize=9)
for row in range(1, 3):
  ylabel = r'$\alpha$ = ' + str(alphas[row])
  ylabel = ylabel + '\n\n'
  if row == 1:
    ylabel = ylabel + 'Testing loss (lower is better)'
  axes[row][0].set_ylabel(ylabel, fontsize=9)
_ = axes[0][0].set_ylabel('No smoothing\n\n', fontsize=9)
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.savefig('synthetic.pdf', bbox_inches = 'tight')

In [0]:
a = df.query('situation == "Similar task"').query('lam == 0.3').query('mode == "Multitask"')['test_loss']
b = df.query('situation == "Similar task"').query('lam == 1').query('mode == "Projection"').query('alpha == 0.01').query('overall == True')['test_loss']
result = permutation_test(a, b, func='x_mean > y_mean', method='approximate', num_rounds=100000, seed=0)
print('p-value for "Similar task": %.6f%%'% (100*result))

a = df.query('situation == "Same task"').query('lam == 1').query('mode == "Multitask"')['test_loss']
b = df.query('situation == "Same task"').query('lam == 3').query('mode == "Projection"').query('alpha == 0.01').query('overall == True')['test_loss']
result = permutation_test(a, b, func='x_mean > y_mean', method='approximate', num_rounds=100000, seed=0)
print('p-value for "Same task": %.6f%%'% (100*result))

In [0]:
def remove_useless_zero(x):
  return x if x[-2:] != '.0' else x[:-2]

results_file = "experiment-2.csv"
df_all = pd.read_csv(results_file)
df = df_all[df_all['overall']]

single_task_df = df[df['mode'] == 'Single task'].copy()
lambdas = list(np.unique(df['lam']))
ticks = [remove_useless_zero(str(x)) for x in lambdas]
for lam in [lambdas[1], lambdas[-1]]:
  single_task_df.loc[:, 'lam'] = lam
  df = pd.concat([df, single_task_df], ignore_index=True)
df = df[df['lam'] > 0]

hue_order = [x for x in order if x in np.unique(df['mode'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])

def chart_synthetic(df, legend='brief', hue='mode', ax=None):
  if ax is None:
    ax = plt.gca()

  sns.lineplot(x='lam',
               y='test_loss',
               style=hue,
               style_order=hue_order,
               hue=hue,
               hue_order=hue_order,
               data=df,
               legend=legend,
               ax=ax)
  ax.set_xscale('log')
  ax.xaxis.set_major_locator(matplotlib.ticker.FixedLocator(lambdas))
  ax.xaxis.set_minor_locator(matplotlib.ticker.NullLocator())
  ax.xaxis.set_major_formatter(matplotlib.ticker.FixedFormatter(ticks))
  ax.tick_params(axis='both', which='major', labelsize=9)
  sns.despine()

fig, axes = plt.subplots(figsize=(6.75, 3.5), ncols=4, nrows=3, sharey=True, sharex=True)
axes[0][1].set_title('Similar task', fontsize=9)

situations = ['Unrelated task', 'Similar task', 'Same task']
alphas = list(np.unique(df['alpha']))[::-1]

df1 = df.query('mode == "Single task"')
for column in range(3):
  df2 = pd.concat([df1, df.query('mode == "Multitask"').query('situation == "' + situations[column] + '"')])
  for row in range(3):  
    df3 = pd.concat([df2, df.query('mode != "Multitask"')
                            .query('mode != "Single task"')
                            .query('situation == "' + situations[column] + '"')
                            .query('alpha == '+str(alphas[row]))])
    legend = False if ((row < 2) | (column < 2)) else 'brief'
    chart_synthetic(df3, legend=legend, ax=axes[row][column])
handles, labels = axes[2][2].get_legend_handles_labels()
axes[2][2].clear()
chart_synthetic(df3, legend=False, ax=axes[2][2])
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
_ = axes[1][3].legend(handles[1:], labels[1:], loc='center', frameon=False, fontsize=9)
for row in range(3):
  axes[row][3].axis('off')
for column in range(3):
  axes[0][column].set_title(situations[column], fontsize=9)
  _ = axes[2][column].set_xlabel(r'$\lambda$ (log scale)' if column == 1 else '', fontsize=9)
for row in range(1, 3):
  ylabel = r'$\alpha$ = ' + str(alphas[row])
  ylabel = ylabel + '\n\n'
  if row == 1:
    ylabel = ylabel + 'Testing loss (lower is better)'
  axes[row][0].set_ylabel(ylabel, fontsize=9)
_ = axes[0][0].set_ylabel('No smoothing\n\n', fontsize=9)
plt.subplots_adjust(wspace=0.1, hspace=0.1)
plt.savefig('synthetic3.pdf', bbox_inches = 'tight')

In [0]:
a = df_all.query('situation == "Similar task"').query('lam == 0.3').query('mode == "Orthogonal"').query('alpha == 0.1').query('overall == True')['test_loss']
b = df_all.query('situation == "Similar task"').query('lam == 1').query('mode == "Projection"').query('alpha == 0.01').query('overall == True')['test_loss']
result = permutation_test(a, b, func='x_mean < y_mean', method='approximate', num_rounds=100000, seed=0)
print('p-value for "Similar task": %.6f%%'% (100*result))

a = df_all.query('situation == "Same task"').query('lam == 0.3').query('mode == "Orthogonal"').query('alpha == 0.1').query('overall == True')['test_loss']
b = df_all.query('situation == "Same task"').query('lam == 3').query('mode == "Projection"').query('alpha == 0.01').query('overall == True')['test_loss']
result = permutation_test(a, b, func='x_mean > y_mean', method='approximate', num_rounds=100000, seed=0)
print('p-value for "Same task": %.6f%%'% (100*result))

In [0]:
df_all.pivot_table(index=['mode', 'alpha'], columns=['lam', 'situation', 'overall'], values='test_loss', aggfunc=len).applymap(lambda x: int(np.nan_to_num(x)))

#Experiment 3: NLP

In [0]:
df = pd.read_csv('experiment-3.csv')

df = df[df['mode'] != 'Unweighted cosine']
df = df[df['mode'] != 'Weighted cosine']
df = df[df['mode'] != 'Orthogonal']

df['mode2'] = df['mode']
for i in df.index:
  if df.loc[i, 'mode'] == 'Projection':
    if df.loc[i, 'alpha'] == 1:
      df.loc[i, 'mode2'] = r'Projection ($\alpha$ = 1)'
    elif df.loc[i, 'alpha'] == 0.01:
      df.loc[i, 'mode2'] = r'Projection ($\alpha$ = 0.01)'

lambdas = list(np.sort(np.unique(df['lam'])))
single_task_df = df[df['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df = pd.concat([df, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df = pd.concat([df, single_task_df], ignore_index=True)

hue_order = [x for x in order if x in np.unique(df['mode2'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])
sns.lineplot(x="lam", y="accuracy", hue='mode2', hue_order=hue_order, style='mode2', style_order=hue_order, data=df.query('lam > 0'))
plt.xscale('log')
matplotlib.pyplot.gcf().set_size_inches(3.25, 3.25)
ax = plt.axes()
handles, labels = ax.get_legend_handles_labels()
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.legend(handles[1:], labels[1:], frameon=False, fontsize=9)
ax.tick_params(axis='both', which='major', labelsize=9)
plt.ylabel('Testing accuracy (higher is better)', fontsize=9)
plt.xlabel(r'$\lambda$ (log scale)', fontsize=9)
sns.despine()
plt.savefig('nlp.pdf', bbox_inches = 'tight')

In [0]:
a = df.query('lam == 0.3').query('mode == "Multitask"')['accuracy']
b = df.query('lam == 1').query('mode == "Projection"').query('alpha == 0.01')['accuracy']
result = permutation_test(a, b, func='x_mean < y_mean', method='approximate', num_rounds=100000, seed=0)
print('%.1f%%'% (100*result))

In [0]:
df = pd.read_csv('experiment-3.csv')

palette = sns.color_palette("tab10", n_colors=10)
sns.set_palette([palette[1], palette[0], palette[2], palette[3], palette[4], palette[5]])

df['mode'] = df['mode'].replace('Weighted', 'Weighted cosine').replace('Unweighted', 'Unweighted cosine')
df_all = df

lambdas = list(np.sort(np.unique(df['lam'])))
single_task_df = df[df['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df = pd.concat([df, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df = pd.concat([df, single_task_df], ignore_index=True)

hue_order = [x for x in order if x in np.unique(df['mode'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])
fig, axes = plt.subplots(figsize=(6.75, 3), ncols=3, sharey=True)

for i in range(2):
  alpha = 1 if i == 0 else 0.01
  df2 = df.query(f'(alpha=={alpha}) or (mode=="Single task" or mode=="Multitask") ')
  ax = axes[i]
  sns.lineplot(x="lam", ax=ax, y="accuracy", hue='mode', hue_order=hue_order, style='mode', style_order=hue_order, data=df2.query('lam > 0'), legend=False)
  ax.set_xscale('log')
  ax.tick_params(axis='both', which='major', labelsize=9)
  ax.set_ylabel('Testing accuracy (higher is better)', fontsize=9)
  ax.set_xlabel(r'$\lambda$ (log scale)', fontsize=9)
  ax.set_title('No smoothing' if alpha == 1 else r'$\alpha$ = 0.01', fontsize=9)
  sns.despine()
  plt.ylim((0.85, 0.875))

ax = axes[2]
sns.lineplot(x="lam", ax=ax, y="accuracy", hue='mode', hue_order=hue_order, style='mode', style_order=hue_order, data=df2.query('lam > 0'))
handles, labels = ax.get_legend_handles_labels()
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.clear()
ax.axis('off')
ax.legend(handles[1:], labels[1:], frameon=False, fontsize=9, loc='center')

plt.savefig('nlp2.pdf', bbox_inches = 'tight')

In [0]:
a = df.query('lam == 1').query('mode == "Projection"').query('alpha == 0.01')['accuracy']
b = df.query('lam == 1').query('mode == "Orthogonal"').query('alpha == 0.01')['accuracy']
result = permutation_test(a, b, func='x_mean < y_mean', method='approximate', num_rounds=100000, seed=0)
print('%.1f%%'% (100*result))

In [0]:
a = df.query('lam == 3').query('mode == "Weighted cosine"').query('alpha == 1')['accuracy']
b = df.query('lam == 1').query('mode == "Orthogonal"').query('alpha == 0.01')['accuracy']
result = permutation_test(a, b, func='x_mean < y_mean', method='approximate', num_rounds=100000, seed=0)
print('%.1f%%'% (100*result))

In [0]:
df_all.pivot_table(index=['mode', 'alpha'], columns='lam', values='run', aggfunc=len).applymap(lambda x: int(np.nan_to_num(x)))

#Experiment 4: Weight decay

In [0]:
results_file = 'experiment-4.csv'

df_all = pd.read_csv(results_file)
df = df_all
df = df[df['mode'] != 'Unweighted cosine']
df = df[df['mode'] != 'Weighted cosine']
df = df[df['mode'] != 'Orthogonal']
lambdas = list(np.sort(np.unique(df['lam'])))
single_task_df = df[df['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df = pd.concat([df, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df = pd.concat([df, single_task_df], ignore_index=True)

df['mode2'] = df['mode']
for i in df.index:
  if df.loc[i, 'mode'] == 'Projection':
    if df.loc[i, 'alpha'] == 1:
      df.loc[i, 'mode2'] = r'Projection ($\alpha$ = 1)'
    elif df.loc[i, 'alpha'] == 0.01:
      df.loc[i, 'mode2'] = r'Projection ($\alpha$ = 0.01)'

matplotlib.pyplot.gcf().set_size_inches(3.25, 3.25)
ax = plt.axes()

hue_order = [x for x in order if x in np.unique(df['mode2'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])
sns.lineplot(x="lam", ax=ax, y="loss", hue='mode2', hue_order=hue_order, style='mode2', style_order=hue_order, data=df.query('lam > 0'))

plt.xscale('log')
ax.tick_params(axis='both', which='major', labelsize=9)
plt.ylabel('Testing cross-entropy loss (lower is better)', fontsize=9)
plt.xlabel(r'$\lambda$ (log scale)', fontsize=9)
sns.despine()
plt.ylim((0.29,0.4))
handles, labels = ax.get_legend_handles_labels()
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.legend(handles[1:], labels[1:], frameon=False, fontsize=9)
plt.savefig('wd.pdf', bbox_inches = 'tight')


In [0]:
a = df.query('lam == 30').query('mode == "Multitask"')['loss']
b = df.query('lam == 100').query('mode == "Projection"').query('alpha == 0.01')['loss']
result = permutation_test(a, b, func='x_mean > y_mean', method='approximate', num_rounds=100000, seed=0)
print('%.6f%%'% (100*result))

In [0]:
results_file = 'experiment-4.csv'

df_all = pd.read_csv(results_file)

lambdas = list(np.sort(np.unique(df_all['lam'])))
single_task_df = df_all[df_all['lam'] == 0].copy()
single_task_df['mode'] = 'Single task'
single_task_df.loc[:, 'lam'] = lambdas[1]
df_all = pd.concat([df_all, single_task_df], ignore_index=True)
single_task_df.loc[:, 'lam'] = lambdas[-1]
df_all = pd.concat([df_all, single_task_df], ignore_index=True)

hue_order = [x for x in order if x in np.unique(df_all['mode'])]
sns.set_palette([color_code[x.split(' ')[0]] for x in hue_order])
fig, axes = plt.subplots(figsize=(6.75, 3), ncols=3, sharey=True)

for i in range(2):
  alpha = 1 if i == 0 else 0.01
  df = df_all.query(f'(alpha=={alpha}) or (mode=="Single task" or mode=="Multitask") ')
  ax = axes[i]
  sns.lineplot(x="lam", ax=ax, y="loss", hue='mode', hue_order=hue_order, style='mode', style_order=hue_order, data=df.query('lam > 0'), legend=False)
  ax.set_xscale('log')
  ax.tick_params(axis='both', which='major', labelsize=9)
  ax.set_ylabel('Testing cross-entropy loss (lower is better)', fontsize=9)
  ax.set_xlabel(r'$\lambda$ (log scale)', fontsize=9)
  ax.set_title('No smoothing' if alpha == 1 else r'$\alpha$ = 0.01', fontsize=9)
  sns.despine()
  plt.ylim((0.295,0.35))

ax = axes[2]
sns.lineplot(x="lam", ax=ax, y="loss", hue='mode', hue_order=hue_order, style='mode', style_order=hue_order, data=df.query('lam > 0'))
handles, labels = ax.get_legend_handles_labels()
labels = ['Multi-task' if s == 'Multitask' else s for s in labels]
ax.clear()
ax.axis('off')
ax.legend(handles[1:], labels[1:], frameon=False, fontsize=9, loc='center')

plt.savefig('wd2.pdf', bbox_inches = 'tight')

In [0]:
a = df_all.query('lam == 100').query('mode == "Orthogonal"').query('alpha == 0.01')['loss']
b = df_all.query('lam == 100').query('mode == "Projection"').query('alpha == 0.01')['loss']
result = permutation_test(a, b, func='x_mean > y_mean', method='approximate', num_rounds=100000, seed=0)
print('%.1f%%'% (100*result))

In [0]:
df_all.pivot_table(index=['mode', 'alpha'], columns='lam', values='run', aggfunc=len).applymap(lambda x: int(np.nan_to_num(x)))